### Refining the data before curation

#### Costumers

O que é Particionamento no Spark?
É uma técnica de organização de dados que:

Divide fisicamente os arquivos por valores de colunas

Cria subpastas automáticas no formato nome_coluna=valor

Melhora performance em consultas filtradas por essas colunas

In [3]:
# spark_config.py
import os
from pyspark.sql import SparkSession

def get_spark_session(app_name):
    """Configuração compartilhada por todos os scripts"""
    # 1. Configuração do Hadoop para Windows
    hadoop_path = "C:\\hadoop"
    os.environ['HADOOP_HOME'] = hadoop_path
    os.environ['PATH'] = f"{os.environ['PATH']};{hadoop_path}\\bin"
    os.environ['HADOOP_USER_NAME'] = "user"
    
    # 2. Configurações essenciais do Spark
    return SparkSession.builder \
        .appName(app_name) \
        .config("spark.driver.extraJavaOptions", "-Dio.netty.tryReflectionSetAccessible=true") \
        .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
        .config("spark.sql.parquet.compression.codec", "snappy") \
        .getOrCreate()

In [2]:
# customer_refinement_fixed.py
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import os

# 1. Configuração de caminhos ABSOLUTOS
BASE_DIR = os.path.dirname(os.path.abspath(os.getcwd()))  # Volta um nível para o projeto
INPUT_PATH = os.path.join(BASE_DIR, "data_lake", "parquet", "brazilian_ecommerce_v4", "olist_customers_dataset")
OUTPUT_PATH = os.path.join(BASE_DIR, "data_lake", "refined", "customers")  # Pasta definitiva

# 2. Função de transformação
def transform_customers(df):
    # Limpeza
    df_clean = (df
        .withColumn("customer_city_clean", lower(trim(col("customer_city"))))
        .withColumn("customer_state_clean", upper(trim(col("customer_state"))))
        .dropDuplicates(["customer_id"])
    )
    
    # Cria região (exemplo simplificado)
    return df_clean.withColumn(
        "customer_region",
        when(col("customer_state_clean").isin(["SP", "RJ", "MG", "ES"]), "Sudeste")
        .when(col("customer_state_clean").isin(["RS", "SC", "PR"]), "Sul")
        .when(col("customer_state_clean").isin(["MT", "MS", "GO", "DF"]), "Centro-Oeste")
        .when(col("customer_state_clean").isin(["AM", "PA", "AC", "RO", "RR", "AP", "TO"]), "Norte")
        .when(col("customer_state_clean").isin(["BA", "SE", "AL", "PE", "PB", "RN", "CE", "PI", "MA"]), "Nordeste")
        .otherwise("Outros")  # Apenas para casos extremos não mapeados
    )

# 3. Execução principal
if __name__ == "__main__":
    spark = SparkSession.builder \
        .appName("CustomerRefinement") \
        .config("spark.sql.sources.partitionOverwriteMode", "dynamic") \
        .getOrCreate()

    try:
        # Garante que a pasta de output existe
        os.makedirs(OUTPUT_PATH, exist_ok=True)
        
        print("📂 Lendo dados de:", INPUT_PATH)
        customers_df = spark.read.parquet(INPUT_PATH)
        
        print("🔄 Transformando dados...")
        customers_refined = transform_customers(customers_df)
        
        print("💾 Salvando em:", OUTPUT_PATH)
        (customers_refined.write
            .mode("overwrite")
            .partitionBy("customer_region", "customer_state_clean")  # Partição dupla
            .parquet(OUTPUT_PATH))
        
        # Verificação
        print("✅ Estrutura criada:")
        for root, dirs, files in os.walk(OUTPUT_PATH):
            print(f"📁 {root.replace(BASE_DIR, '...')}")
            for dir in dirs:
                if "=" in dir:  # Mostra apenas pastas de partição
                    print(f"   └── {dir}")

    except Exception as e:
        print(f"❌ Erro: {str(e)}")
    finally:
        spark.stop()

📂 Lendo dados de: c:\Users\pacie\Desktop\dataplatform-pipelines\data_lake\parquet\brazilian_ecommerce_v4\olist_customers_dataset
❌ Erro: An error occurred while calling o62.parquet.
: java.lang.UnsatisfiedLinkError: 'boolean org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(java.lang.String, int)'
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:817)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1415)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1620)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:739)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2078)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2122)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:961)
	at org.apache.spark.util.HadoopFSUtils$.listLeafFiles(HadoopFSUtils.scala:218)
	at org.apache.spark.util.HadoopFSUtil

### Geolocation

In [21]:
# geolocation_refinement.py
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import os

# Configuração de caminhos
BASE_DIR = os.path.dirname(os.path.abspath(os.getcwd()))  # Volta um nível
INPUT_PATH = os.path.join(BASE_DIR, "data_lake", "parquet", "brazilian_ecommerce_v4", "olist_geolocation_dataset")
OUTPUT_PATH = os.path.join(BASE_DIR, "data_lake", "refined", "geolocation")

def transform_geolocation_data(df):
    """Transforma os dados de geolocalização com:
    - Limpeza de textos
    - Categorização por região
    - Controle de qualidade
    """
    # Limpeza e padronização
    df_clean = (df
        .withColumn("geolocation_city_clean", lower(trim(col("geolocation_city"))))
        .withColumn("geolocation_state_clean", upper(trim(col("geolocation_state"))))
        .na.drop(subset=["geolocation_zip_code_prefix"])  # Remove registros sem CEP
    )
    
    # Cria região geográfica (mesma lógica dos clientes)
    df_with_region = df_clean.withColumn(
        "region",
        when(col("geolocation_state_clean").isin(["SP", "RJ", "MG", "ES"]), "Sudeste")
        .when(col("geolocation_state_clean").isin(["RS", "SC", "PR"]), "Sul")
        .when(col("geolocation_state_clean").isin(["MT", "MS", "GO", "DF"]), "Centro-Oeste")
        .when(col("geolocation_state_clean").isin(["AM", "PA", "AC", "RO", "RR", "AP", "TO"]), "Norte")
        .otherwise("Nordeste")
    )
    
    # Cria ponto geográfico (lat+lng)
    return df_with_region.withColumn(
        "geo_point",
        format_string("POINT(%s %s)", col("geolocation_lng"), col("geolocation_lat"))
    )

if __name__ == "__main__":
    spark = SparkSession.builder.appName("GeolocationRefinement").getOrCreate()
    
    try:
        # Leitura
        print("📍 Lendo dados de geolocalização...")
        geo_df = spark.read.parquet(INPUT_PATH)
        
        # Transformação
        print("🛠️ Processando dados...")
        geo_refined = transform_geolocation_data(geo_df)
        
        # Escrita com particionamento por região/estado
        print("💾 Salvando em:", OUTPUT_PATH)
        (geo_refined.write
            .mode("overwrite")
            .partitionBy("region", "geolocation_state_clean")
            .option("compression", "snappy")
            .parquet(OUTPUT_PATH))
        
        # Verificação
        print("✅ Dados salvos! Estrutura:")
        print(f"Total de registros: {geo_refined.count()}")
        print("\nAmostra dos dados transformados:")
        geo_refined.show(3)
        
        print("\nExemplo de estrutura gerada:")
        for region in ["Sudeste", "Nordeste", "Sul"]:
            path = os.path.join(OUTPUT_PATH, f"region={region}")
            if os.path.exists(path):
                print(f"📁 {path}")
                print(f"   Estados: {os.listdir(path)}")

    except Exception as e:
        print(f"❌ Falha no processamento: {str(e)}")
        if 'geo_df' in locals():
            print("\nEsquema original:", geo_df.printSchema())
    finally:
        spark.stop()

📍 Lendo dados de geolocalização...
🛠️ Processando dados...
💾 Salvando em: c:\Users\pacie\Desktop\Projeto A\data_lake\refined\geolocation
✅ Dados salvos! Estrutura:
Total de registros: 1000163

Amostra dos dados transformados:
+---------------------------+------------------+-------------------+----------------+-----------------+----------------------+-----------------------+--------+--------------------+
|geolocation_zip_code_prefix|   geolocation_lat|    geolocation_lng|geolocation_city|geolocation_state|geolocation_city_clean|geolocation_state_clean|  region|           geo_point|
+---------------------------+------------------+-------------------+----------------+-----------------+----------------------+-----------------------+--------+--------------------+
|                      50760|-8.072588074383969| -34.91359211518264|          recife|               PE|                recife|                     PE|Nordeste|POINT(-34.9135921...|
|                      50740|-8.042627210658422|-3

### Order Items

In [1]:
# order_items_refinement.py
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import os

def get_spark_session(app_name):
    """
    Configuração compartilhada por todos os scripts, com opções para Hadoop.
    
    Verifica a presença de 'winutils.exe' para evitar 'UnsatisfiedLinkError' no Windows.
    """
    # 1. Configuração do Hadoop para Windows (ajuste se necessário para outros SOs)
    # ATENÇÃO: Verifique o caminho para o seu diretório Hadoop e baixe o binário
    # do winutils.exe compatível com a sua versão do Spark/Hadoop.
    hadoop_path = "C:\\hadoop"
    winutils_path = os.path.join(hadoop_path, "bin", "winutils.exe")
    
    # Adicionando debug para verificar os caminhos
    print(f"✔️ Verificando HADOOP_HOME em: {hadoop_path}")
    print(f"✔️ Verificando winutils.exe em: {winutils_path}")
    
    # Verifica se o winutils.exe existe antes de tentar configurar as variáveis de ambiente
    if not os.path.exists(winutils_path):
        raise FileNotFoundError(
            f"❌ Erro: O arquivo winutils.exe não foi encontrado em '{winutils_path}'. "
            f"Isso geralmente indica um problema com a variável de ambiente HADOOP_HOME "
            f"ou que o binário necessário não está no local correto. "
            f"Por favor, verifique se a pasta '{hadoop_path}' contém a estrutura de pastas "
            f"'bin' e se o 'winutils.exe' está dentro dela."
        )
        
    os.environ['HADOOP_HOME'] = hadoop_path
    os.environ['PATH'] = f"{os.environ['PATH']};{hadoop_path}\\bin"
    os.environ['HADOOP_USER_NAME'] = "user"
    
    # 2. Configurações essenciais do Spark
    return SparkSession.builder \
        .appName(app_name) \
        .config("spark.driver.extraJavaOptions", "-Dio.netty.tryReflectionSetAccessible=true") \
        .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
        .config("spark.sql.parquet.compression.codec", "snappy") \
        .getOrCreate()


# Configuração de caminhos
BASE_DIR = os.path.dirname(os.path.abspath(os.getcwd()))
INPUT_PATH = os.path.join(BASE_DIR, "data_lake", "parquet", "brazilian_ecommerce_v4", "olist_order_items_dataset")
OUTPUT_PATH = os.path.join(BASE_DIR, "data_lake", "refined", "order_items")

def transform_order_items(df):
    """Transformações principais para order_items"""
    # 1. Limpeza básica
    df_clean = df.withColumn(
        "shipping_limit_date", 
        to_timestamp(col("shipping_limit_date"))
    ).na.drop(subset=["order_id", "product_id", "seller_id"])
    
    # 2. Cálculo de valores totais
    df_with_totals = df_clean.withColumn(
        "total_value", 
        round(col("price") + col("freight_value"), 2)
    )
    
    # 3. Categorização por faixa de preço
    df_with_price_range = df_with_totals.withColumn(
        "price_range",
        when(col("price") < 50, "0-50")
        .when((col("price") >= 50) & (col("price") < 100), "50-100")
        .when((col("price") >= 100) & (col("price") < 150), "100-150") 
        .otherwise("150+")
    )
    
    # 4. Categorização por frete
    df_with_freight_category = df_with_price_range.withColumn(
        "freight_ratio",
        round(col("freight_value") / col("price"), 2)
    ).withColumn(
        "freight_category",
        when(col("freight_ratio") < 0.1, "Baixo")
        .when((col("freight_ratio") >= 0.1) & (col("freight_ratio") < 0.25), "Médio")
        .otherwise("Alto")
    )
    
    # 5. Adição de período do dia (baseado no horário do shipping)
    return df_with_freight_category.withColumn(
        "shipping_time_period",
        when(hour(col("shipping_limit_date")).between(6, 11), "Manhã")
        .when(hour(col("shipping_limit_date")).between(12, 17), "Tarde")
        .when(hour(col("shipping_limit_date")).between(18, 23), "Noite")
        .otherwise("Madrugada")
    )

if __name__ == "__main__":
    # Inicia a sessão Spark usando a função de configuração
    spark = get_spark_session("OrderItemsRefinement")
    
    try:
        # Leitura
        print("📦 Lendo dados de order_items...")
        order_items_df = spark.read.parquet(INPUT_PATH)
        
        # Transformação
        print("🛠️ Enriquecendo dados...")
        order_items_refined = transform_order_items(order_items_df)
        
        # Escrita com particionamento
        print("💾 Salvando dados refinados...")
        (order_items_refined.write
            .mode("overwrite")
            .partitionBy("price_range", "freight_category")  # Particionamento estratégico
            .parquet(OUTPUT_PATH))
        
        # Verificação
        print("✅ Processo concluído! Amostra dos dados:")
        order_items_refined.show(5)
        
        print("\n📊 Estatísticas:")
        order_items_refined.select(
            mean("price").alias("avg_price"),
            mean("freight_value").alias("avg_freight"),
            countDistinct("order_id").alias("unique_orders")
        ).show()
        
    except Exception as e:
        print(f"❌ Erro: {str(e)}")
    finally:
        spark.stop()


✔️ Verificando HADOOP_HOME em: C:\hadoop
✔️ Verificando winutils.exe em: C:\hadoop\bin\winutils.exe
📦 Lendo dados de order_items...
🛠️ Enriquecendo dados...
💾 Salvando dados refinados...
✅ Processo concluído! Amostra dos dados:
+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+-----------+-----------+-------------+----------------+--------------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date| price|freight_value|total_value|price_range|freight_ratio|freight_category|shipping_time_period|
+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+-----------+-----------+-------------+----------------+--------------------+
|8ac26cb701a7887cc...|            1|4ebb87ba41ca44632...|7a67c85e85bb2ce85...|2017-05-22 16:05:14|109.99|        18.02|     128.01|    100-150|         0.16|           Médio|          

### Order payments


In [2]:
# order_payments_refinement.py
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import os

def get_spark_session(app_name):
    """
    Configuração compartilhada por todos os scripts, com opções para Hadoop.
    
    Verifica a presença de 'winutils.exe' para evitar 'UnsatisfiedLinkError' no Windows.
    """
    # 1. Configuração do Hadoop para Windows (ajuste se necessário para outros SOs)
    hadoop_path = "C:\\hadoop"
    winutils_path = os.path.join(hadoop_path, "bin", "winutils.exe")
    
    # Adicionando debug para verificar os caminhos
    print(f"✔️ Verificando HADOOP_HOME em: {hadoop_path}")
    print(f"✔️ Verificando winutils.exe em: {winutils_path}")
    
    # Verifica se o winutils.exe existe antes de tentar configurar as variáveis de ambiente
    if not os.path.exists(winutils_path):
        raise FileNotFoundError(
            f"❌ Erro: O arquivo winutils.exe não foi encontrado em '{winutils_path}'. "
            f"Por favor, verifique se a pasta '{hadoop_path}' contém a estrutura de pastas "
            f"'bin' e se o 'winutils.exe' está dentro dela."
        )
        
    os.environ['HADOOP_HOME'] = hadoop_path
    os.environ['PATH'] = f"{os.environ['PATH']};{hadoop_path}\\bin"
    os.environ['HADOOP_USER_NAME'] = "user"
    
    # 2. Configurações essenciais do Spark
    return SparkSession.builder \
        .appName(app_name) \
        .config("spark.driver.extraJavaOptions", "-Dio.netty.tryReflectionSetAccessible=true") \
        .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
        .config("spark.sql.parquet.compression.codec", "snappy") \
        .getOrCreate()

# Configuração de caminhos
BASE_DIR = os.path.dirname(os.path.abspath(os.getcwd()))
INPUT_PATH = os.path.join(BASE_DIR, "data_lake", "parquet", "brazilian_ecommerce_v4", "olist_order_payments_dataset")
OUTPUT_PATH = os.path.join(BASE_DIR, "data_lake", "refined", "order_payments")

def transform_order_payments(df):
    """Transformações principais para order_payments"""
    # 1. Limpeza básica
    df_clean = df.na.drop(subset=["order_id", "payment_type", "payment_value"])
    
    # 2. Categorização de tipos de pagamento
    df_with_payment_categories = df_clean.withColumn(
        "payment_type_category",
        when(col("payment_type").isin(["credit_card", "debit_card"]), "cartão")
        .when(col("payment_type") == "boleto", "boleto")
        .when(col("payment_type") == "voucher", "voucher")
        .otherwise("outros")
    )
    
    # 3. Categorização por valor de pagamento
    df_with_value_ranges = df_with_payment_categories.withColumn(
        "payment_value_range",
        when(col("payment_value") < 50, "0-50")
        .when((col("payment_value") >= 50) & (col("payment_value") < 100), "50-100")
        .when((col("payment_value") >= 100) & (col("payment_value") < 200), "100-200")
        .otherwise("200+")
    )
    
    # 4. Categorização por parcelamento
    df_with_installment_categories = df_with_value_ranges.withColumn(
        "installment_category",
        when(col("payment_installments") == 1, "à vista")
        .when((col("payment_installments") > 1) & (col("payment_installments") <= 6), "parcelado_curto")
        .when((col("payment_installments") > 6) & (col("payment_installments") <= 12), "parcelado_médio")
        .otherwise("parcelado_longo")
    )
    
    # 5. Flag para pagamentos com valores discrepantes
    return df_with_installment_categories.withColumn(
        "is_high_value_payment",
        when(col("payment_value") > 500, True).otherwise(False)
    )

if __name__ == "__main__":
    # Inicia a sessão Spark usando a função de configuração
    spark = get_spark_session("OrderPaymentsRefinement")
    
    try:
        # Leitura
        print("📦 Lendo dados de order_payments...")
        order_payments_df = spark.read.parquet(INPUT_PATH)
        
        # Transformação
        print("🛠️ Enriquecendo dados...")
        order_payments_refined = transform_order_payments(order_payments_df)
        
        # Escrita com particionamento
        print("💾 Salvando dados refinados...")
        (order_payments_refined.write
            .mode("overwrite")
            .partitionBy("payment_type_category", "payment_value_range")  # Particionamento estratégico
            .parquet(OUTPUT_PATH))
        
        # Verificação
        print("✅ Processo concluído! Amostra dos dados:")
        order_payments_refined.show(5)
        
        print("\n📊 Estatísticas:")
        order_payments_refined.select(
            count("*").alias("total_payments"),
            mean("payment_value").alias("avg_payment"),
            countDistinct("order_id").alias("unique_orders"),
            sum(when(col("is_high_value_payment") == True, 1).otherwise(0)).alias("high_value_payments")
        ).show()
        
    except Exception as e:
        print(f"❌ Erro: {str(e)}")
    finally:
        spark.stop()

✔️ Verificando HADOOP_HOME em: C:\hadoop
✔️ Verificando winutils.exe em: C:\hadoop\bin\winutils.exe
📦 Lendo dados de order_payments...
🛠️ Enriquecendo dados...
💾 Salvando dados refinados...
✅ Processo concluído! Amostra dos dados:
+--------------------+------------------+------------+--------------------+-------------+---------------------+-------------------+--------------------+---------------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|payment_type_category|payment_value_range|installment_category|is_high_value_payment|
+--------------------+------------------+------------+--------------------+-------------+---------------------+-------------------+--------------------+---------------------+
|b81ef226f3fe1789b...|                 1| credit_card|                   8|        99.33|               cartão|             50-100|     parcelado_médio|                false|
|a9810da82917af2d9...|                 1| credit_card|               

In [19]:
import os
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# ==========================================
# CONFIGURAÇÕES
# ==========================================
INPUT_PATH = r"c:\Users\pacie\Desktop\Projeto A\data_lake\parquet\brazilian_ecommerce_v4\olist_order_reviews_dataset"
OUTPUT_PATH = r"c:\Users\pacie\Desktop\Projeto A\data_lake\refined\order_reviews"
HADOOP_HOME = r"C:\hadoop"

os.environ["HADOOP_HOME"] = HADOOP_HOME
os.environ["PATH"] += os.pathsep + os.path.join(HADOOP_HOME, "bin")

# ==========================================
# INICIALIZAÇÃO DO SPARK
# ==========================================
spark = SparkSession.builder \
    .appName("OrderReviewsRefinement") \
    .getOrCreate()

print(f"📦 Lendo dados de: {INPUT_PATH}")
df = spark.read.parquet(INPUT_PATH)

# ==========================================
# LIMPEZA
# ==========================================
df = df.withColumn("review_comment_title", F.trim(F.col("review_comment_title"))) \
       .withColumn("review_comment_message", F.trim(F.col("review_comment_message")))

# Datas como timestamp (tolerante a erro)
df = df.withColumn("review_creation_ts", F.to_timestamp("review_creation_date", "yyyy-MM-dd HH:mm:ss")) \
       .withColumn("review_answer_ts", F.to_timestamp("review_answer_timestamp", "yyyy-MM-dd HH:mm:ss"))

# ==========================================
# SCORE NUMÉRICO COM TRY_CAST
# ==========================================
df = df.withColumn("review_score_int", F.expr("try_cast(review_score as int)"))

# ==========================================
# ENRIQUECIMENTO
# ==========================================
df = df.withColumn(
    "review_score_category",
    F.when(F.col("review_score_int") >= 4, "positive")
     .when(F.col("review_score_int") == 3, "neutral")
     .when(F.col("review_score_int") <= 2, "negative")
     .otherwise(None)
)

df = df.withColumn(
    "sentiment",
    F.when(F.col("review_score_int") >= 4, "positivo")
     .when(F.col("review_score_int") == 3, "neutro")
     .when(F.col("review_score_int") <= 2, "negativo")
     .otherwise(None)
)

df = df.withColumn(
    "has_comment",
    F.when(F.col("review_comment_message").isNotNull() & (F.col("review_comment_message") != ""), True)
     .otherwise(False)
)

df = df.withColumn(
    "response_time_hours",
    (F.unix_timestamp("review_answer_ts") - F.unix_timestamp("review_creation_ts")) / 3600
)

df = df.withColumn(
    "response_speed_category",
    F.when(F.col("response_time_hours") <= 24, "fast")
     .when((F.col("response_time_hours") > 24) & (F.col("response_time_hours") <= 72), "medium")
     .when(F.col("response_time_hours") > 72, "slow")
     .otherwise(None)
)

# ==========================================
# SALVAR
# ==========================================
print("💾 Salvando dados refinados...")
df.write.mode("overwrite").partitionBy("review_score_category", "sentiment").parquet(OUTPUT_PATH)

print(f"✅ Processo concluído! Dados salvos em {OUTPUT_PATH}")
spark.stop()


📦 Lendo dados de: c:\Users\pacie\Desktop\Projeto A\data_lake\parquet\brazilian_ecommerce_v4\olist_order_reviews_dataset
💾 Salvando dados refinados...


DateTimeException: [CANNOT_PARSE_TIMESTAMP] Unparseable date: " produto ainda não chegou ao destino.". Use `try_to_timestamp` to tolerate invalid input string and return NULL instead. SQLSTATE: 22007

### Orders Dataset

In [ ]:
# orders_refinement.py
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.functions import countDistinct
from pyspark.sql.types import *
import os

def get_spark_session(app_name):
    """
    Configuração compartilhada por todos os scripts, com opções para Hadoop.
    """
    hadoop_path = "C:\\hadoop"
    winutils_path = os.path.join(hadoop_path, "bin", "winutils.exe")
    
    print(f"✔️ Verificando HADOOP_HOME em: {hadoop_path}")
    print(f"✔️ Verificando winutils.exe em: {winutils_path}")
    
    if not os.path.exists(winutils_path):
        raise FileNotFoundError(
            f"❌ Erro: winutils.exe não encontrado em '{winutils_path}'. "
            f"Verifique se o Hadoop está corretamente instalado."
        )
        
    os.environ['HADOOP_HOME'] = hadoop_path
    os.environ['PATH'] = f"{os.environ['PATH']};{hadoop_path}\\bin"
    os.environ['HADOOP_USER_NAME'] = "user"
    
    return SparkSession.builder \
        .appName(app_name) \
        .config("spark.driver.extraJavaOptions", "-Dio.netty.tryReflectionSetAccessible=true") \
        .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
        .config("spark.sql.parquet.compression.codec", "snappy") \
        .getOrCreate()

# Configuração de caminhos
BASE_DIR = os.path.dirname(os.path.abspath(os.getcwd()))
INPUT_PATH = os.path.join(BASE_DIR, "data_lake", "parquet", "brazilian_ecommerce_v4", "olist_orders_dataset")
OUTPUT_PATH = os.path.join(BASE_DIR, "data_lake", "refined", "orders")

def transform_orders(df):
    """Transformações principais para orders"""
    # 1. Limpeza básica e conversão de datas
    df_clean = df.withColumn(
        "order_purchase_timestamp", 
        to_timestamp(col("order_purchase_timestamp"), "yyyy-MM-dd HH:mm:ss")
    ).withColumn(
        "order_approved_at",
        to_timestamp(col("order_approved_at"), "yyyy-MM-dd HH:mm:ss")
    ).withColumn(
        "order_delivered_carrier_date",
        to_timestamp(col("order_delivered_carrier_date"), "yyyy-MM-dd HH:mm:ss")
    ).withColumn(
        "order_delivered_customer_date",
        to_timestamp(col("order_delivered_customer_date"), "yyyy-MM-dd HH:mm:ss")
    ).withColumn(
        "order_estimated_delivery_date",
        to_timestamp(col("order_estimated_delivery_date"), "yyyy-MM-dd HH:mm:ss")
    ).na.drop(subset=["order_id", "customer_id", "order_status"])
    
    # 2. Cálculo de tempos de processamento
    df_with_times = df_clean.withColumn(
        "approval_time_hours",
        round((col("order_approved_at").cast("long") - col("order_purchase_timestamp").cast("long")) / 3600,
        2
    )).withColumn(
        "processing_time_hours",
        round((col("order_delivered_carrier_date").cast("long") - col("order_approved_at").cast("long")) / 3600,
        2
    )).withColumn(
        "shipping_time_hours",
        round((col("order_delivered_customer_date").cast("long") - col("order_delivered_carrier_date").cast("long")) / 3600,
        2
    )).withColumn(
        "total_delivery_time_hours",
        round((col("order_delivered_customer_date").cast("long") - col("order_purchase_timestamp").cast("long")) / 3600,
        2
    )).withColumn(
        "delivery_delay_hours",
        round((col("order_delivered_customer_date").cast("long") - col("order_estimated_delivery_date").cast("long")) / 3600,
        2
    ))
    
    # 3. Categorização de status
    df_with_status = df_with_times.withColumn(
        "status_category",
        when(col("order_status") == "delivered", "Entregue")
        .when(col("order_status") == "shipped", "Enviado")
        .when(col("order_status") == "canceled", "Cancelado")
        .when(col("order_status") == "unavailable", "Indisponível")
        .otherwise("Outro")
    )
    
    # 4. Categorização de desempenho de entrega
    df_with_delivery = df_with_status.withColumn(
        "delivery_performance",
        when(col("delivery_delay_hours").isNull(), "Não entregue")
        .when(col("delivery_delay_hours") <= 0, "No prazo ou antecipado")
        .when(col("delivery_delay_hours") <= 24, "Pequeno atraso (<24h)")
        .when(col("delivery_delay_hours") <= 72, "Atraso moderado (24-72h)")
        .otherwise("Grande atraso (>72h)")
    )
    
    # 5. Categorização por período do dia (compra)
    return df_with_delivery.withColumn(
        "purchase_time_period",
        when(hour(col("order_purchase_timestamp")).between(6, 11), "Manhã")
        .when(hour(col("order_purchase_timestamp")).between(12, 17), "Tarde")
        .when(hour(col("order_purchase_timestamp")).between(18, 23), "Noite")
        .otherwise("Madrugada")
    )

if __name__ == "__main__":
    spark = get_spark_session("OrdersRefinement")
    
    try:
        # Leitura
        print("📦 Lendo dados de orders...")
        orders_df = spark.read.parquet(INPUT_PATH)
        
        # Transformação
        print("🛠️ Enriquecendo dados...")
        orders_refined = transform_orders(orders_df)
        
        # Escrita com particionamento
        print("💾 Salvando dados refinados...")
        (orders_refined.write
            .mode("overwrite")
            .partitionBy("status_category", "delivery_performance")  # Particionamento estratégico
            .parquet(OUTPUT_PATH))
        
        # Verificação
        print("✅ Processo concluído! Amostra dos dados:")
        orders_refined.show(5)
        
        print("\n📊 Estatísticas:")
        orders_refined.select(
            count("*").alias("total_orders"),
            countDistinct("customer_id").alias("unique_customers"),
            mean("total_delivery_time_hours").alias("avg_delivery_time_hours"),
            mean("delivery_delay_hours").alias("avg_delay_hours"),
            sum(when(col("delivery_performance") == "No prazo ou antecipado", 1).otherwise(0)).alias("on_time_deliveries")
        ).show()
        
    except Exception as e:
        print(f"❌ Erro: {str(e)}")
    finally:
        spark.stop()

### Products Dataset 

In [14]:
# products_refinement.py
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import os

def get_spark_session(app_name):
    """Configuração do Spark com Hadoop"""
    hadoop_path = "C:\\hadoop"
    os.environ['HADOOP_HOME'] = hadoop_path
    os.environ['PATH'] = f"{os.environ['PATH']};{hadoop_path}\\bin"
    
    return SparkSession.builder \
        .appName(app_name) \
        .config("spark.sql.parquet.compression.codec", "snappy") \
        .getOrCreate()

def transform_products(df):
    """Transformações principais para products"""
    # 1. Limpeza básica
    df_clean = df.na.drop(subset=["product_id", "product_category_name"]) \
        .fillna({
            'product_name_lenght': 0,
            'product_description_lenght': 0,
            'product_photos_qty': 1,
            'product_weight_g': 0,
            'product_length_cm': 0,
            'product_height_cm': 0,
            'product_width_cm': 0
        })
    
    # 2. Cálculo de volume e densidade
    df_enriched = df_clean.withColumn(
        "product_volume_cm3",
        round(col("product_length_cm") * col("product_height_cm") * col("product_width_cm"), 2)
    ).withColumn(
        "product_density_g_cm3",
        when(col("product_volume_cm3") > 0, 
             round(col("product_weight_g") / col("product_volume_cm3"), 4))
        .otherwise(None)
    )
    
    # 3. Categorizações
    return df_enriched.withColumn(
        "size_category",
        when(col("product_weight_g") < 100, "Pequeno")
        .when((col("product_weight_g") >= 100) & (col("product_weight_g") < 500), "Médio")
        .when((col("product_weight_g") >= 500) & (col("product_weight_g") < 2000), "Grande")
        .otherwise("Muito Grande")
    ).withColumn(
        "photo_category",
        when(col("product_photos_qty") == 0, "Sem foto")
        .when(col("product_photos_qty") == 1, "Uma foto")
        .when(col("product_photos_qty") <= 3, "Poucas fotos")
        .otherwise("Muitas fotos")
    ).withColumn(
        "description_category",
        when(col("product_description_lenght") == 0, "Sem descrição")
        .when(col("product_description_lenght") < 50, "Descrição curta")
        .when(col("product_description_lenght") < 200, "Descrição média")
        .otherwise("Descrição longa")
    )

if __name__ == "__main__":
    spark = get_spark_session("ProductsRefinement")
    
    # Configuração de caminhos CORRIGIDA para Parquet
    BASE_DIR = os.path.dirname(os.path.abspath(os.getcwd()))
    INPUT_PATH = os.path.join(BASE_DIR, "data_lake", "parquet", "brazilian_ecommerce_v4", "olist_products_dataset")
    OUTPUT_PATH = os.path.join(BASE_DIR, "data_lake", "refined", "products")
    
    try:
        # Leitura do Parquet CORRIGIDA
        print("📦 Lendo dados de products (Parquet)...")
        products_df = spark.read.parquet(INPUT_PATH)
        
        # Transformação
        print("🛠️ Enriquecendo dados...")
        products_refined = transform_products(products_df)
        
        # Escrita com particionamento
        print("💾 Salvando dados refinados...")
        (products_refined.write
            .mode("overwrite")
            .partitionBy("size_category", "photo_category")
            .parquet(OUTPUT_PATH))
        
        # Verificação
        print("✅ Processo concluído! Amostra dos dados:")
        products_refined.show(5)
        
        print("\n📊 Estatísticas:")
        products_refined.select(
            count("*").alias("total_products"),
            countDistinct("product_category_name").alias("unique_categories"),
            mean("product_weight_g").alias("avg_weight_g"),
            mean("product_volume_cm3").alias("avg_volume_cm3"),
            mean("product_density_g_cm3").alias("avg_density")
        ).show()
        
    except Exception as e:
        print(f"❌ Erro: {str(e)}")
    finally:
        spark.stop()

📦 Lendo dados de products (Parquet)...
🛠️ Enriquecendo dados...


💾 Salvando dados refinados...
✅ Processo concluído! Amostra dos dados:
+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+------------------+---------------------+-------------+--------------+--------------------+
|          product_id|product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|product_volume_cm3|product_density_g_cm3|size_category|photo_category|description_category|
+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+------------------+---------------------+-------------+--------------+--------------------+
|1e9e8ef04dbcff454...|           perfumaria|                 40|                       287|                 1|   